# import libraries and trained model

In [ ]:
import os
from os import listdir
from os.path import isfile, join
import numpy as np
from keras.models import load_model
from skimage.io import imread

# define constants

In [ ]:
IM_PATH = './ec/' # path containing all input images
OUT_PATH = './Color_results/' # output path
os.mkdir(OUT_PATH) #if folder is already present, then comment out this line
NUM_CLASSES = 4
model = load_model('ecDNA_model.h5')

# Crop into patches and stitch back together

In [ ]:
onlyfiles = [f for f in listdir(IM_PATH) if isfile(join(IM_PATH, f))]
for i in onlyfiles:
    img = IM_PATH + i
    img = imread(img)
    print(i)
    name = OUT_PATH +i
    crops = []
    y = 1
    x = 1
    dim = 256
    shape = img.shape
    vcrop = int(shape[0]/256)
    hcrop = int(shape[1]/256)
    #image[70:170, 440:540]
    for a in range(0,5):
        y = 1
        for k in range(0,4):
            train = img[y:y+dim, x:x+dim]
            crops.append(train)
            y = y+dim
        x = x+dim
    pred = []   
    for i in range(0,len(crops)):
        x = np.expand_dims(crops[i], axis=0)
        comb_pred = np.squeeze(model.predict(x, verbose=0))
        pred.append(comb_pred)
    stitched_im = np.ones((256*vcrop,1,NUM_CLASSES))
    index = -1
    for j in range (1,hcrop+1):
        index = index +1
        if(index >=hcrop*vcrop):
            break
        row = pred[index]
        for k in range(1,vcrop):
            index = index + 1
            I = pred[index]
            row = np.vstack((row, I))
        stitched_im = np.hstack((stitched_im, row))
    stitched_im = stitched_im[:, 1:, :]
    np.save(name, stitched_im)